In [1]:
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
import seaborn as sns
from IPython.display import display
from matplotlib.ticker import MaxNLocator
import random

# 参数
m = 15
K = 5
L_levels = [20, 15, 10, 5, 1]  
E_levels = [13, 28, 38, 45, 120] 
E_PU_max = [5500] * m # limit 650
CPU_capacity = [600] * m
f_v_levels = [15, 20, 30, 40, 60]
P_idle = [18] * m
P_max = [36] * m
T_total = 300
total_tasks = 100


chain_tasks = {
    0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    1: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
    2: [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
    3: [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
    4: [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
    5: [50, 51, 52, 53, 54, 55, 56, 57],
    6: [58, 59, 60],
    7: [61, 62, 63],
    8: [64, 65, 66],
    9: [67, 68, 69],
    10: [70, 71, 72],
    11: [73, 74, 75],
    12: [76, 77, 78],
    13: [79, 80, 81],
    14: [82, 83, 84],
    15: [85, 86, 87],
    16: [88, 89, 90],
    17: [91, 92, 93],
    18: [94, 95, 96],
    19: [97, 98, 99]
}


num_chains = len(chain_tasks)
n = max(max(v) for v in chain_tasks.values()) + 1  # 总任务数
L_max = [len(chain_tasks[k]) * 15 for k in chain_tasks]  # 每条链的最大延迟




In [2]:
# ✅ 增强 Greedy 实现：保持与 Gurobi 相同的约束，包括系统总资源限制 C_total_max

def run_greedy_model(params, chain_enabled=None):
    alpha_1, alpha_2, alpha_pu, E_levels, L_levels, f_v_levels, CPU_capacity, \
    P_idle, P_max, T_total, chain_tasks, L_max, E_PU_max, n, m = params

    delta = [0] * n
    y = [-1] * n
    w = [-1] * n

    pu_remaining_cpu = CPU_capacity.copy()
    pu_workload = [0] * m
    maf_energy = 0
    accepted_chains = 0

    for k in range(len(chain_tasks)):
        if chain_enabled and chain_enabled.get(k, 1) == 0:
            continue

        tasks = chain_tasks[k]
        task_levels = []
        task_cpus = []
        chain_latency = 0

        # 为任务链内每个任务贪心选择最低满足延迟的能耗等级
        feasible = True
        for i in tasks:
            selected_level = -1
            for level in range(K):
                if chain_latency + L_levels[level] <= L_max[k]:
                    selected_level = level
                    chain_latency += L_levels[level]
                    break
            if selected_level == -1:
                feasible = False
                break
            task_levels.append(selected_level)
            task_cpus.append(f_v_levels[selected_level])

        if not feasible:
            continue  # 延迟约束不可满足

        # 选择PU（根据CPU剩余量与能耗最优）
        best_pu = -1
        min_energy_increase = float('inf')

        for j in range(m):
            if pu_remaining_cpu[j] >= sum(task_cpus):
                predicted_workload = pu_workload[j] + sum(L_levels[lv] for lv in task_levels)
                idle_time = max(0, T_total - predicted_workload)
                pu_e = P_idle[j] * idle_time + (P_max[j] - P_idle[j]) * predicted_workload
                energy_increase = pu_e - (P_idle[j] * max(0, T_total - pu_workload[j]) + (P_max[j] - P_idle[j]) * pu_workload[j])

                if energy_increase < min_energy_increase and pu_e <= E_PU_max[j]:
                    min_energy_increase = energy_increase
                    best_pu = j

        if best_pu == -1:
            continue  # 没有合适PU

        # 安排任务到PU
        for idx, i in enumerate(tasks):
            delta[i] = 1
            y[i] = task_levels[idx]
            w[i] = best_pu
            pu_remaining_cpu[best_pu] -= task_cpus[idx]
            pu_workload[best_pu] += L_levels[task_levels[idx]]
            maf_energy += E_levels[task_levels[idx]]

        accepted_chains += 1

    # 计算PU总能耗
    pu_energy = sum(
        P_idle[j] * max(0, T_total - pu_workload[j]) + (P_max[j] - P_idle[j]) * pu_workload[j]
        for j in range(m)
    )

    total_energy = maf_energy + pu_energy
    acceptance_rate = accepted_chains / len(chain_tasks)
    objective = alpha_1 * accepted_chains - alpha_2 * maf_energy - alpha_pu * pu_energy

    return {
        "delta": delta,
        "w": w,
        "y": y,
        "objective": objective,
        "total_energy": total_energy,
        "acceptance_rate": acceptance_rate,
        "accepted_chains": accepted_chains,
        "maf_energy": maf_energy,
        "pu_energy": pu_energy
    }


In [4]:
params = [
    14000,  # alpha_1
    10,      # alpha_2
    1,      # alpha_pu
    E_levels,
    L_levels,
    f_v_levels,
    CPU_capacity,
    P_idle,
    P_max,
    T_total,
    chain_tasks,
    L_max,
    E_PU_max,
    n,
    m
]

result_greedy = run_greedy_model(params)

print("📌 修复后的贪心算法执行结果:")
print(f"目标函数值: {result_greedy['objective']}")
print(f"接受的任务链数量: {result_greedy['accepted_chains']} / {len(chain_tasks)}")
print(f"接受率: {result_greedy['acceptance_rate']:.1%}")
print(f"总能耗: {result_greedy['total_energy']}")
print(f"任务能耗: {result_greedy['maf_energy']}")
print(f"PU能耗: {result_greedy['pu_energy']}")



📌 修复后的贪心算法执行结果:
目标函数值: 105060
接受的任务链数量: 14 / 20
接受率: 70.0%
总能耗: 81994
任务能耗: 994
PU能耗: 81000
